In [ ]:
import numpy as np
import tensorflow as tf

# Exercises

TensorFlow is basically a gradient descent optimization tool and deep learning is just a special case of optimization. As such, let's try out some simple optimization problems and in the process gain more familiarity with the low-level tools. Try to implement the following using gradient descent:

## Exercise 1: make one value as close as possible to another
Specify a target value (say, 2), and a variable x (initialized to, say, 18) and minimize the distance between x and the target. This task is as simple as it sounds.

## Exercise 2: find the mean of vector of numbers
The mean of a vector of numbers is value that minimizes the sum of the square distances to all those values. It just so happens that there's an analytic solution to this optimization problem, so it's never solved using optimization. However, just for practice, let's find it using gradient descent!

## Exercise 3: invert a square matrix
Create a random nxn matrix and use TensorFlow to find the inverse. Construct a scalar loss function, and then minimize it. Then, compare it with the result of directly computing the inverse (e.g., with `np.linalg.inv`). 

## Exercise 4: find the eigendecomposition of an nxn matrix
This is also an introduction to using complex numbers in tensorflow (`tf.complex64`), an uncommon use case, but will be useful later when we model optical fields, which are complex-valued.

## Exercise 1: make one value as close as possible to another

In [ ]:
target = tf.constant(2, dtype=tf.float32)
x = tf.Variable(18, dtype=tf.float32)
loss = (x-target)**2 
# loss = tf.abs(x-target)

# you can reuse this for each exercise (but play around with the learning_rate):
train_op = tf.train.GradientDescentOptimizer(learning_rate=.1).minimize(loss)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(1000):
    sess.run(train_op)
    print(sess.run(x))
    

## Exercise 2: find the mean of vector of numbers

In [ ]:
vec = np.random.randn(1000)+10  # mean is 10
mean = tf.Variable(0, dtype=np.float32)
loss = tf.reduce_sum((mean-vec)**2)

In [ ]:
train_op = tf.train.GradientDescentOptimizer(.0001).minimize(loss)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(50):
    loss_i, mean_i, _ = sess.run([loss, mean, train_op])
    print(mean_i)

## Exercise 3: invert a square matrix

In [ ]:
n = 5
matrix = np.random.randn(n, n).astype(np.float32)
inverse = tf.get_variable(name='inverse',shape=(n, n), dtype=tf.float32, initializer=tf.random_normal_initializer)
identity_pred = tf.matmul(matrix, inverse)
loss = tf.reduce_mean((identity_pred-np.eye(5))**2)

In [ ]:
train_op = tf.train.GradientDescentOptimizer(1.).minimize(loss)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
losses = list()
for i in range(10000):
    loss_i, pred_i, _ = sess.run([loss, identity_pred, train_op])
    losses.append(loss_i)
    if i %1000 == 0:
        print(loss_i)

In [ ]:
# compare:
print(inverse.eval())
print(np.linalg.inv(matrix))

## Exercise 4: find the eigendecomposition of an nxn matrix

In [ ]:
n = 5
A = np.random.randn(n, n).astype(np.float32)

Q_re = tf.get_variable(name='Q_real',shape=(n, n), dtype=tf.float32, initializer=tf.random_normal_initializer)
Q_im = tf.get_variable(name='Q_imag',shape=(n, n), dtype=tf.float32, initializer=tf.random_normal_initializer)
Q = tf.complex(Q_re, Q_im)

eigs_re = tf.get_variable(name='eigenvalue_real', shape=(n), initializer=tf.ones_initializer)
eigs_im = tf.get_variable(name='eigenvalue_imag', shape=(n), initializer=tf.ones_initializer)
eigs = tf.complex(eigs_re, eigs_im)
S = tf.linalg.tensor_diag(eigs)

factorization = Q @ S @ tf.linalg.inv(Q) # prediction for A
loss = tf.reduce_mean(tf.abs(A-factorization)**2)

In [ ]:
train_op = tf.train.GradientDescentOptimizer(.1).minimize(loss)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
losses = list()
# if you get a noninvertible error, try decreasing your learning rate
for i in range(20000):
    loss_i, _ = sess.run([loss, train_op])
    losses.append(loss_i)
    if i %1000 == 0:
        print(loss_i)

In [ ]:
# what are the optimized eigenvalues?
# I normalize them by their magnitude to make it easier to compare to direct calculation
# alternatively, we can enforce normalization in the tf graph
eigs_pred = eigs.eval()
eigs_pred /= np.abs(eigs_pred)
print(np.sort(eigs_pred))


In [ ]:
# compare to direct calculation:
eigs_truth, _ = np.linalg.eig(A)
eigs_truth /= np.abs(eigs_truth)
print(np.sort(eigs_truth))